<a href="https://colab.research.google.com/github/AlphaKermit-999/AI-Topics-learnings/blob/main/Advanced_AI/Autoencoder_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd

df = pd.read_csv("anomaly.csv")

In [42]:
df.head()

,timestamp,value,is_anomaly,predicted
0,1425008573,42,False,44.072500
1,1425008873,41,False,50.709390
2,1425009173,41,False,81.405120
3,1425009473,61,False,39.950367
4,1425009773,44,False,35.350160


In [43]:
df["date"] = pd.to_datetime(df["timestamp"], unit='s')
df

,timestamp,value,is_anomaly,predicted,date
0,1425008573,42,False,44.072500,2015-02-27 03:42:53
1,1425008873,41,False,50.709390,2015-02-27 03:47:53
2,1425009173,41,False,81.405120,2015-02-27 03:52:53
3,1425009473,61,False,39.950367,2015-02-27 03:57:53
4,1425009773,44,False,35.350160,2015-02-27 04:02:53
...,...,...,...,...,...
15825,1429756073,44,False,53.624115,2015-04-23 02:27:53
15826,1429756373,45,False,59.752296,2015-04-23 02:32:53
15827,1429756673,48,False,52.147630,2015-04-23 02:37:53
15828,1429756973,26,False,58.007545,2015-04-23 02:42:53


In [44]:
df["date"].dt.year.unique()


array([2015], dtype=int32)

In [45]:
df["date"].dt.month.unique()

array([2, 3, 4], dtype=int32)

In [46]:
df.head()

,timestamp,value,is_anomaly,predicted,date
0,1425008573,42,False,44.072500,2015-02-27 03:42:53
1,1425008873,41,False,50.709390,2015-02-27 03:47:53
2,1425009173,41,False,81.405120,2015-02-27 03:52:53
3,1425009473,61,False,39.950367,2015-02-27 03:57:53
4,1425009773,44,False,35.350160,2015-02-27 04:02:53


In [47]:
df.drop(["date", "timestamp"], axis=1, inplace=True)

In [48]:
x = df.drop("is_anomaly", axis=1)
y=df["is_anomaly"]

In [49]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [50]:
y_train[y_train==False].index

Index([ 7589,  5674, 12546, 13647, 13556,  6818, 10041, 13936, 14571,  5168,
       ...
        6265,  5734, 11284, 11964,  5191, 13418,  5390,   860, 15795,  7270],
      dtype='int64', length=12057)

In [51]:
train_index = y_train[y_train==False].index
normal = x_train.loc[train_index]

In [52]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense

In [53]:
class Autoencoder(Model):
  def __init__(self, output_units, encoding_size):
    super().__init__()

    self.encoder = Sequential([
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(16, activation="relu"),
        Dense(encoding_size, activation="relu")
    ])
    self.decoder = Sequential([
        Dense(16, activation="relu"),
        Dense(32, activation="relu"),
        Dense(64, activation="relu"),
        Dense(output_units, activation="sigmoid")
    ])

  def call(self, inputs):
    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded

In [54]:
model = Autoencoder(output_units=x_train.shape[1], encoding_size=2)

In [55]:
model.compile(loss="msle", metrics=["mse"], optimizer="adam")

In [56]:
model.fit(x_train, x_train, epochs=10, batch_size=32, validation_data=(x_test, x_test))

Epoch 1/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 11.4768 - mse: 49057.1328 - val_loss: 11.1501 - val_mse: 113864.6250
Epoch 2/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.1643 - mse: 52764.8711 - val_loss: 11.1501 - val_mse: 113864.6250
Epoch 3/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 11.1494 - mse: 41671.1719 - val_loss: 11.1501 - val_mse: 113864.6250
Epoch 4/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 11.1996 - mse: 43538.1875 - val_loss: 11.1501 - val_mse: 113864.6250
Epoch 5/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 11.2857 - mse: 45609.2734 - val_loss: 11.1500 - val_mse: 113864.6250
Epoch 6/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 11.2256 - mse: 65683.6719 - val_loss: 11.1500 - val_mse: 113864.6250
Epoch 7/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 11.2985 - mse: 51626.6367 - val_loss: 11.1500 - val_mse: 113864.6250
Epoch 8/10
396/396 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 11.2362 - mse: 48449.5000 - val_loss:

In [57]:
import tensorflow as tf
import numpy as np

In [58]:
def find_threshold(model, data):
  pred = model.predict(data)
  error = tf.keras.losses.msle(pred, data)
  threshold = np.mean(error)+np.std(error)
  return threshold

In [59]:
def predictions(model, xtest, threshold):
  pred = model.predict(xtest)
  error = tf.keras.losses.msle(pred, xtest)
  anomaly = pd.Series(error)>threshold
  return anomaly, error

In [60]:
threshold = find_threshold(model, x_train)

396/396 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [61]:
pred = predictions(model, x_test, threshold)

99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [62]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(pred[0], y_test))

              precision    recall  f1-score   support

       False       0.91      0.97      0.94      2823
        True       0.43      0.21      0.28       343

    accuracy                           0.88      3166
   macro avg       0.67      0.59      0.61      3166
weighted avg       0.86      0.88      0.87      3166



The idea is we are going to check where anomaly is currently available, observe its data and then return back to our original data for comparison.